In [1]:
import pandas as pd
import numpy as np
import itertools
import random
from tqdm import tqdm

pd.set_option('display.max_columns', 50)

In [2]:
reg_data_m =pd.read_csv('data/MRegularSeasonDetailedResults.csv')
reg_data_m.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14


In [3]:
ncaa_data_m = pd.read_csv('data/MNCAATourneyDetailedResults.csv')
ncaa_data_m.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,11,29,17,26,14,30,17,12,5,3,22,29,67,12,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,7,23,11,14,11,36,22,16,10,7,8,20,64,4,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,6,14,16,22,10,27,18,9,7,4,19,25,69,7,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,3,7,18,25,11,20,15,18,13,1,19,27,60,7,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,7,20,15,23,18,20,17,13,8,2,14,25,56,9,21,15,20,10,26,16,14,5,8,19


In [4]:
#removing WLoc
print(reg_data_m['WLoc'].value_counts())
print(ncaa_data_m['WLoc'].value_counts())
# removing WLoc because the final data to predicted on will only have it as Neutral so not needed

H    63049
A    33019
N    10766
Name: WLoc, dtype: int64
N    1248
Name: WLoc, dtype: int64


In [5]:
# thinking of removing NumOT
print("Regular Season\n",reg_data_m['NumOT'].value_counts())
print("NCAA\n",ncaa_data_m['NumOT'].value_counts())
# since NumOT is generally 0 and it is 1 for very few samples in the NCAA data, i believe it to not contribute a whole lot

Regular Season
 0    100790
1      5004
2       844
3       157
4        33
5         5
6         1
Name: NumOT, dtype: int64
NCAA
 0    1170
1      65
2      13
Name: NumOT, dtype: int64


In [6]:
reg_data_w =pd.read_csv('data/WRegularSeasonDetailedResults.csv')
reg_data_w.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,11,3103,63,3237,49,H,0,23,54,5,9,12,19,10,26,14,18,7,0,15,20,54,3,13,6,10,11,27,11,23,7,6,19
1,2010,11,3104,73,3399,68,N,0,26,62,5,12,16,28,16,31,15,20,5,2,25,25,63,4,21,14,27,14,26,7,20,4,2,27
2,2010,11,3110,71,3224,59,A,0,29,62,6,15,7,12,14,23,18,13,6,2,17,19,58,2,14,19,23,17,23,8,15,6,0,15
3,2010,11,3111,63,3267,58,A,0,27,52,4,11,5,9,6,40,14,27,5,10,18,18,74,6,26,16,25,22,22,15,11,14,5,14
4,2010,11,3119,74,3447,70,H,1,30,74,7,20,7,11,14,33,18,11,5,3,18,25,74,9,17,11,21,21,32,12,14,4,2,14


In [7]:
ncaa_data_w = pd.read_csv('data/WNCAATourneyDetailedResults.csv')
ncaa_data_w.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,138,3124,69,3201,55,N,0,28,57,1,5,12,19,13,24,22,12,6,2,12,21,61,10,34,3,5,17,19,12,18,4,1,18
1,2010,138,3173,67,3395,66,N,0,23,59,9,26,12,19,13,34,13,16,3,10,14,22,73,8,27,14,15,18,26,8,8,8,6,22
2,2010,138,3181,72,3214,37,H,0,26,57,4,13,16,22,13,34,15,11,10,7,11,15,56,4,15,3,8,10,21,4,16,6,4,20
3,2010,138,3199,75,3256,61,H,0,25,63,3,15,22,26,20,27,13,17,8,3,21,21,62,2,20,17,22,16,21,13,16,5,4,24
4,2010,138,3207,62,3265,42,N,0,24,68,8,25,6,8,20,29,16,8,5,5,18,13,60,5,26,11,17,16,22,9,10,3,4,12


In [8]:
#removing WLoc
print(reg_data_w['WLoc'].value_counts())
print(ncaa_data_w['WLoc'].value_counts())
# removing WLoc because the final data to predicted on will only have it as Neutral so not needed

H    38742
A    25171
N     6094
Name: WLoc, dtype: int64
N    492
H    228
A     40
Name: WLoc, dtype: int64


In [9]:
# thinking of removing NumOT
print("Regular Season\n",reg_data_w['NumOT'].value_counts())
print("NCAA\n",ncaa_data_w['NumOT'].value_counts())
# since NumOT is generally 0 and it is 1 for very few samples in the NCAA data, i believe it to not contribute a whole lot

Regular Season
 0    66911
1     2633
2      397
3       54
4       11
5        1
Name: NumOT, dtype: int64
NCAA
 0    734
1     23
2      3
Name: NumOT, dtype: int64


In [10]:
data = pd.concat([reg_data_m,ncaa_data_m,reg_data_w,ncaa_data_w])
print(data.shape)
print(data['Season'].value_counts())

(178849, 34)
2019    10833
2018    10744
2017    10735
2014    10718
2016    10708
2015    10698
2013    10593
2022    10539
2020    10499
2011    10460
2012    10433
2010    10427
2023     9400
2021     7540
2009     5313
2008     5227
2007     5107
2006     4821
2005     4739
2003     4680
2004     4635
Name: Season, dtype: int64


In [11]:
drop_cols = ['WLoc','NumOT']
reg_data = data.drop(columns = drop_cols)
cols = reg_data.columns.values
print(cols)
reg_data.head()

['Season' 'DayNum' 'WTeamID' 'WScore' 'LTeamID' 'LScore' 'WFGM' 'WFGA'
 'WFGM3' 'WFGA3' 'WFTM' 'WFTA' 'WOR' 'WDR' 'WAst' 'WTO' 'WStl' 'WBlk'
 'WPF' 'LFGM' 'LFGA' 'LFGM3' 'LFGA3' 'LFTM' 'LFTA' 'LOR' 'LDR' 'LAst'
 'LTO' 'LStl' 'LBlk' 'LPF']


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14


In [12]:
# converting it to team A and team B from WTeam and LTeam
list_reg_data = []

for row in tqdm(reg_data.itertuples()):
    new_reg_data = dict()
    coin = random.random()
    new_reg_data['Season'] = row.Season
    new_reg_data['DayNum'] = row.DayNum

    if coin>0.5:
        new_reg_data['A'] = row.WTeamID
        new_reg_data['B'] = row.LTeamID
        try:
            new_reg_data['FGP_A'] = (row.WFGM / row.WFGA) * 100
        except:
            new_reg_data['FGP_A'] = 0
        try:
            new_reg_data['FGP3_A'] = (row.WFGM3 / row.WFGA3) * 100
        except:
            new_reg_data['FGP3_A'] = 0
        try:
            new_reg_data['FTP_A'] = (row.WFTM / row.WFTA) * 100
        except:
            new_reg_data['FTP_A'] = 0
        #new_reg_data['OR_A'] = row.WOR
        #new_reg_data['DR_A'] = row.WDR
        try:
            new_reg_data['AST_TO_A'] = row.WAst/row.WTO
        except:
            new_reg_data['AST_TO_A'] = row.WAst/(row.WTO+1)
        #new_reg_data['STL_A'] = row.WStl
        new_reg_data['BLK_A'] = row.WBlk
        new_reg_data['PF_A'] = row.WPF
        new_reg_data['PM_A'] = row.WScore - row.LScore # plus minus for team A
        new_reg_data['POSS_A'] = row.WFGA - row.WOR + row.WTO + (0.475*row.WFTA) # https://thepowerrank.com/cbb-analytics/ possession
        new_reg_data['REB_DIFF_A'] = row.WOR - row.WDR
        try:
            new_reg_data['FGP_B'] = (row.LFGM / row.LFGA) * 100
        except:
            new_reg_data['FGP_B'] = 0
        try:
            new_reg_data['FGP3_B'] = (row.LFGM3 / row.LFGA3) * 100
        except:
            new_reg_data['FGP3_B'] = 0
        try:
            new_reg_data['FTP_B'] = (row.LFTM / row.LFTA) * 100
        except:
            new_reg_data['FTP_B'] = 0
        #new_reg_data['OR_B'] = row.LOR
        #new_reg_data['DR_B'] = row.LDR
        try:
            new_reg_data['AST_TO_B'] = row.LAst/row.LTO
        except:
            new_reg_data['AST_TO_B'] = row.LAst/(row.LTO+1)
        #new_reg_data['STL_B'] = row.LStl
        new_reg_data['BLK_B'] = row.LBlk
        new_reg_data['PF_B'] = row.LPF
        new_reg_data['PM_B'] = row.LScore - row.WScore # plus minus for team B
        new_reg_data['POSS_B'] = row.LFGA - row.LOR + row.LTO + (0.475*row.LFTA)
        new_reg_data['REB_DIFF_B'] = row.LOR - row.LDR
        new_reg_data['Win'] = 1
    else:
        new_reg_data['A'] = row.LTeamID
        new_reg_data['B'] = row.WTeamID
        try:
            new_reg_data['FGP_A'] = (row.LFGM / row.LFGA) * 100
        except:
            new_reg_data['FGP_A'] = 0
        try:
            new_reg_data['FGP3_A'] = (row.LFGM3 / row.LFGA3) * 100
        except:
            new_reg_data['FGP3_A'] = 0
        try:
            new_reg_data['FTP_A'] = (row.LFTM / row.LFTA) * 100
        except:
            new_reg_data['FTP_A'] = 0
        #new_reg_data['OR_A'] = row.LOR
        #new_reg_data['DR_A'] = row.LDR
        try:
            new_reg_data['AST_TO_A'] = row.LAst/row.LTO
        except:
            new_reg_data['AST_TO_A'] = row.LAst/(row.LTO+1)
        #new_reg_data['STL_A'] = row.LStl
        new_reg_data['BLK_A'] = row.LBlk
        new_reg_data['PF_A'] = row.LPF
        new_reg_data['PM_A'] = row.LScore - row.WScore # plus minus for team A
        new_reg_data['POSS_A'] = row.LFGA - row.LOR + row.LTO + (0.475*row.LFTA)
        new_reg_data['REB_DIFF_A'] = row.LOR - row.LDR
        try:
            new_reg_data['FGP_B'] = (row.WFGM / row.WFGA) * 100
        except:
            new_reg_data['FGP_B'] = 0
        try:
            new_reg_data['FGP3_B'] = (row.WFGM3 / row.WFGA3) * 100
        except:
            new_reg_data['FGP3_B'] = 0
        try:
            new_reg_data['FTP_B'] = (row.WFTM / row.WFTA) * 100
        except:
            new_reg_data['FTP_B'] = 0
        #new_reg_data['OR_B'] = row.WOR
        #new_reg_data['DR_B'] = row.WDR
        try:
            new_reg_data['AST_TO_B'] = row.WAst/row.WTO
        except:
            new_reg_data['AST_TO_B'] = row.WAst/(row.WTO+1)
        #new_reg_data['STL_B'] = row.WStl
        new_reg_data['BLK_B'] = row.WBlk
        new_reg_data['PF_B'] = row.WPF
        new_reg_data['PM_B'] = row.WScore - row.LScore # plus minus for team B
        new_reg_data['POSS_B'] = row.WFGA - row.WOR + row.WTO + (0.475*row.WFTA)
        new_reg_data['REB_DIFF_B'] = row.WOR - row.WDR
        new_reg_data['Win'] = 0

    list_reg_data.append(new_reg_data)

new_reg_data = pd.DataFrame(list_reg_data)
new_reg_data


178849it [00:00, 231708.88it/s]


,Season,DayNum,A,B,FGP_A,FGP3_A,FTP_A,AST_TO_A,BLK_A,PF_A,PM_A,POSS_A,REB_DIFF_A,FGP_B,FGP3_B,FTP_B,AST_TO_B,BLK_B,PF_B,PM_B,POSS_B,REB_DIFF_B,Win
0,2003,10,1104,1328,46.551724,21.428571,61.111111,0.565217,1,22,6,75.550,-10,41.509434,20.000000,72.727273,0.444444,2,20,-6,71.450,-12,1
1,2003,10,1272,1393,41.935484,40.000000,52.631579,1.230769,4,18,7,69.025,-13,35.820896,25.000000,45.000000,0.583333,6,16,-7,68.500,-5,1
2,2003,11,1437,1266,30.136986,11.538462,60.869565,0.750000,5,23,-12,64.925,9,41.379310,44.444444,58.620690,1.500000,2,25,12,64.775,-9,0
3,2003,11,1296,1457,47.368421,33.333333,54.838710,0.916667,2,18,6,58.725,-13,36.734694,27.272727,53.333333,0.473684,3,23,-6,58.125,-3,1
4,2003,11,1400,1208,49.180328,42.857143,84.615385,0.857143,4,20,6,64.175,-5,38.709677,37.500000,62.962963,1.200000,1,14,-6,63.825,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178844,2022,147,3163,3301,48.051948,23.809524,60.000000,1.428571,2,16,4,81.500,-11,48.484848,30.434783,84.210526,1.538462,7,16,-4,82.025,-24,1
178845,2022,147,3257,3276,43.103448,33.333333,77.777778,1.090909,4,17,12,67.275,-14,34.782609,21.428571,75.000000,0.428571,2,12,-12,66.500,-14,1
178846,2022,151,3163,3390,36.842105,35.714286,80.000000,0.736842,2,16,5,73.500,-18,34.848485,17.391304,61.538462,0.909091,3,16,-5,72.175,-12,1
178847,2022,151,3376,3257,47.368421,35.294118,70.588235,1.357143,4,11,13,71.075,-16,42.857143,12.500000,57.142857,0.333333,2,17,-13,70.325,-7,1


In [13]:
new_reg_data.iloc[:,4:].describe()

,FGP_A,FGP3_A,FTP_A,AST_TO_A,BLK_A,PF_A,PM_A,POSS_A,REB_DIFF_A,FGP_B,FGP3_B,FTP_B,AST_TO_B,BLK_B,PF_B,PM_B,POSS_B,REB_DIFF_B,Win
count,178849.000000,178849.000000,178849.000000,178849.000000,178849.000000,178849.000000,178849.000000,178849.000000,178849.000000,178849.000000,178849.000000,178849.000000,178849.000000,178849.000000,178849.000000,178849.000000,178849.000000,178849.000000,178849.000000
mean,42.284254,33.064619,69.394681,1.023174,3.301914,17.589939,-0.034163,69.411989,-12.936846,42.297180,33.006990,69.386070,1.025709,3.309261,17.569581,0.034163,69.417293,-12.938468,0.498488
std,7.814938,12.056220,13.095453,0.603236,2.296919,5.062915,16.363110,6.801881,6.640301,7.793787,12.011695,13.101296,0.609464,2.294392,5.061895,16.363110,6.792778,6.655328,0.499999
min,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-108.000000,41.425000,-42.000000,9.259259,0.000000,0.000000,0.000000,0.000000,0.000000,-95.000000,41.800000,-44.000000,0.000000
25%,36.923077,25.000000,61.538462,0.636364,2.000000,15.000000,-11.000000,64.800000,-17.000000,36.956522,25.000000,61.538462,0.636364,2.000000,15.000000,-11.000000,64.800000,-17.000000,0.000000
50%,42.187500,33.333333,70.000000,0.900000,3.000000,18.000000,-1.000000,69.025000,-13.000000,42.222222,33.333333,70.000000,0.900000,3.000000,18.000000,1.000000,69.025000,-13.000000,0.000000
75%,47.540984,40.909091,78.260870,1.266667,5.000000,21.000000,11.000000,73.550000,-9.000000,47.540984,40.909091,78.260870,1.266667,5.000000,21.000000,11.000000,73.550000,-9.000000,1.000000
max,80.000000,100.000000,100.000000,29.000000,20.000000,47.000000,95.000000,126.850000,17.000000,75.806452,100.000000,100.000000,25.000000,21.000000,44.000000,108.000000,128.350000,27.000000,1.000000


In [14]:
new_reg_data['Win'].value_counts()

0    89695
1    89154
Name: Win, dtype: int64

In [15]:
new_reg_data.corr()

,Season,DayNum,A,B,FGP_A,FGP3_A,FTP_A,AST_TO_A,BLK_A,PF_A,PM_A,POSS_A,REB_DIFF_A,FGP_B,FGP3_B,FTP_B,AST_TO_B,BLK_B,PF_B,PM_B,POSS_B,REB_DIFF_B,Win
Season,1.000000,-0.040249,0.290909,0.290646,-0.052992,-0.053244,0.053557,0.046035,-0.008678,-0.166615,-0.000148,0.131212,-0.168289,-0.054883,-0.053785,0.053437,0.046339,-0.013276,-0.167688,0.000148,0.131578,-0.162151,0.000389
DayNum,-0.040249,1.000000,-0.013736,-0.013837,0.025590,0.023968,0.062990,0.063760,-0.016941,-0.050468,-0.000093,-0.154421,-0.013422,0.025315,0.021102,0.060728,0.065748,-0.022180,-0.050065,0.000093,-0.153545,-0.008500,-0.000370
A,0.290909,-0.013736,1.000000,0.988724,-0.241590,-0.129399,-0.007530,-0.150915,-0.008785,-0.083084,0.003504,0.186620,0.029821,-0.244149,-0.129918,-0.007394,-0.154163,-0.010435,-0.079723,-0.003504,0.186286,0.027708,0.001748
B,0.290646,-0.013837,0.988724,1.000000,-0.243527,-0.129303,-0.007253,-0.151687,-0.008370,-0.080994,0.000575,0.186690,0.030289,-0.242113,-0.129434,-0.006292,-0.152774,-0.010110,-0.081017,-0.000575,0.186314,0.026565,-0.001068
FGP_A,-0.052992,0.025590,-0.241590,-0.243527,1.000000,0.530049,0.071850,0.364363,0.102819,-0.049807,0.580692,0.006654,-0.322334,-0.051541,0.001011,0.005428,-0.016227,-0.322197,0.081869,-0.580692,0.022755,0.474886,0.479864
FGP3_A,-0.053244,0.023968,-0.129399,-0.129303,0.530049,1.000000,0.060452,0.271096,0.028404,-0.028536,0.357387,-0.041090,-0.181433,-0.003802,0.009837,0.010653,0.029064,-0.084878,0.007845,-0.357387,-0.016944,0.268079,0.305816
FTP_A,0.053557,0.062990,-0.007530,-0.007253,0.071850,0.060452,1.000000,0.056046,-0.002583,-0.036580,0.156401,0.005299,-0.089450,0.006994,0.012977,0.038627,0.019293,-0.039683,0.018111,-0.156401,0.003309,0.144220,0.137870
AST_TO_A,0.046035,0.063760,-0.150915,-0.151687,0.364363,0.271096,0.056046,1.000000,0.071920,-0.155559,0.424695,-0.114204,-0.076586,-0.015989,0.032868,0.018431,0.045599,-0.093548,-0.071153,-0.424695,-0.104824,-0.040847,0.321504
BLK_A,-0.008678,-0.016941,-0.008785,-0.008370,0.102819,0.028404,-0.002583,0.071920,1.000000,-0.038158,0.233978,0.097647,-0.158204,-0.324178,-0.082865,-0.036380,-0.092025,-0.015187,0.056423,-0.233978,0.108927,0.201300,0.200649
PF_A,-0.166615,-0.050468,-0.083084,-0.080994,-0.049807,-0.028536,-0.036580,-0.155559,-0.038158,1.000000,-0.165298,0.270144,0.096365,0.080398,0.004519,0.020177,-0.071472,0.055114,0.464360,0.165298,0.262654,-0.017942,-0.213900


### Few heavy correlation
- FGP_A - FGP3_A (0.53)
- FGP_A - AST_A (0.55)
- FGP_A - DR_B (negative)
- FGP3_A - AST_A
- TO_A - STL_B (quite high, 0.71)
- STL_A - TO_B (quite high, 0.71)
same for B

In [16]:
def get_avg_team(data, team_id, dn):
    data = data[(data['A'] == team_id)|(data['B'] == team_id)]
    data = data[data['DayNum']<dn]
    final_data = {'FGP_scored':[],'FGP3_scored':[],'FTP_scored':[],
                  'AST_TO_scored':[],'BLK_scored':[],'PF_scored':[],
                  'POSS_scored' : [], 'REB_DIFF_scored':[],
                  'FGP_allowed':[],'FGP3_allowed':[],'FTP_allowed':[],
                  'AST_TO_allowed':[],'BLK_allowed':[],'PF_allowed':[],
                  'POSS_allowed':[], 'REB_DIFF_allowed':[]}
    for row in (data.itertuples()):
        if row.A == team_id:
            final_data['FGP_scored'].append(row.FGP_A)
            final_data['FGP3_scored'].append(row.FGP3_A)
            final_data['FTP_scored'].append(row.FTP_A)
            final_data['AST_TO_scored'].append(row.AST_TO_A)
            final_data['BLK_scored'].append(row.BLK_A)
            final_data['PF_scored'].append(row.PF_A)
            final_data['POSS_scored'].append(row.POSS_A)
            final_data['REB_DIFF_scored'].append(row.REB_DIFF_A)
            final_data['FGP_allowed'].append(row.FGP_B)
            final_data['FGP3_allowed'].append(row.FGP3_B)
            final_data['FTP_allowed'].append(row.FTP_B)
            final_data['AST_TO_allowed'].append(row.AST_TO_B)
            final_data['BLK_allowed'].append(row.BLK_B)
            final_data['PF_allowed'].append(row.PF_B)
            final_data['POSS_allowed'].append(row.POSS_B)
            final_data['REB_DIFF_allowed'].append(row.REB_DIFF_B)
        else:
            final_data['FGP_scored'].append(row.FGP_B)
            final_data['FGP3_scored'].append(row.FGP3_B)
            final_data['FTP_scored'].append(row.FTP_B)
            final_data['AST_TO_scored'].append(row.AST_TO_B)
            final_data['BLK_scored'].append(row.BLK_B)
            final_data['PF_scored'].append(row.PF_B)
            final_data['REB_DIFF_scored'].append(row.REB_DIFF_B)
            final_data['FGP_allowed'].append(row.FGP_A)
            final_data['FGP3_allowed'].append(row.FGP3_A)
            final_data['FTP_allowed'].append(row.FTP_A)
            final_data['AST_TO_allowed'].append(row.AST_TO_A)
            final_data['BLK_allowed'].append(row.BLK_A)
            final_data['PF_allowed'].append(row.PF_A)
            final_data['REB_DIFF_allowed'].append(row.REB_DIFF_A)

    try:
        return [sum(v)/len(v) for v in final_data.values()]
    except:
        return [np.nan]*len(final_data)


def get_avg_yearly(year):
    year_data = new_reg_data[new_reg_data['Season'] == year]
    list_data = []

    for row in tqdm(year_data.itertuples()):
        *stats_A, = get_avg_team(year_data, row.A, row.DayNum)
        *stats_B, = get_avg_team(year_data, row.B, row.DayNum)
        list_data.append([row.Season, row.DayNum, row.A, row.B, *stats_A, *stats_B, row.Win])
    
    data = pd.DataFrame(list_data, columns = [['Season','DayNum','A','B','FGP_SCORED_A','FGP3_SCORED_A','FTP_SCORED_A',
                                               'AST_TO_SCORED_A','BLK_SCORED_A','PF_SCORED_A','POSS_SCORED_A','REB_DIFF_SCORED_A',
                                               'FGP_ALLOWED_A', 'FGP3_ALLOWED_A','FTP_ALLOWED_A','AST_TO_ALLOWED_A',
                                               'BLK_ALLOWED_A','PF_ALLOWED_A','POSS_ALLOWED_A','REF_DIFF_ALLOWED_A','FGP_SCORED_B',
                                               'FGP3_SCORED_B','FTP_SCORED_B','AST_TO_SCORED_B','BLK_SCORED_B','PF_SCORED_B',
                                               'POSS_SCORED_B','REB_DIFF_SCORED_B','FGP_ALLOWED_B', 'FGP3_ALLOWED_B','FTP_ALLOWED_B',
                                               'AST_TO_ALLOWED_B', 'BLK_ALLOWED_B','PF_ALLOWED_B','POSS_ALLOWED_B','REB_DIFF_ALLOWED_B','Win']])
    
    data = data.dropna()

    return data

def get_data_reg():
    year = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
    data = []
    for y in tqdm(year):
        data.append(get_avg_yearly(y))

    final_data = pd.concat(data)
    return final_data


In [17]:
data_03=get_avg_yearly(2003)

4680it [00:11, 417.90it/s]


In [18]:
data_03.iloc[:,4:].describe()

,FGP_SCORED_A,FGP3_SCORED_A,FTP_SCORED_A,AST_TO_SCORED_A,BLK_SCORED_A,PF_SCORED_A,POSS_SCORED_A,REB_DIFF_SCORED_A,FGP_ALLOWED_A,FGP3_ALLOWED_A,FTP_ALLOWED_A,AST_TO_ALLOWED_A,BLK_ALLOWED_A,PF_ALLOWED_A,POSS_ALLOWED_A,REF_DIFF_ALLOWED_A,FGP_SCORED_B,FGP3_SCORED_B,FTP_SCORED_B,AST_TO_SCORED_B,BLK_SCORED_B,PF_SCORED_B,POSS_SCORED_B,REB_DIFF_SCORED_B,FGP_ALLOWED_B,FGP3_ALLOWED_B,FTP_ALLOWED_B,AST_TO_ALLOWED_B,BLK_ALLOWED_B,PF_ALLOWED_B,POSS_ALLOWED_B,REB_DIFF_ALLOWED_B,Win
count,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000,4226.000000
mean,43.850766,34.520769,68.615038,0.979632,3.339276,19.225804,69.969477,-11.334255,43.788702,34.456005,68.345747,0.980396,3.306790,19.215806,69.943852,-11.263292,43.975927,34.568758,68.425429,0.991017,3.335852,19.134642,69.954416,-11.357352,43.715647,34.398177,68.506590,0.978100,3.304822,19.236556,69.915531,-11.242624,0.480833
std,3.418391,4.266783,5.211485,0.235208,1.386572,2.141485,4.550321,2.753472,3.414075,3.975882,4.548372,0.215591,0.961397,2.021853,4.469173,2.602596,3.415657,4.406451,5.387783,0.238149,1.369248,2.059833,4.668492,2.842274,3.390342,3.997071,4.342561,0.212104,0.911509,2.122429,4.557068,2.572244,0.499692
min,21.892077,11.111111,38.616071,0.222222,0.000000,11.000000,48.700000,-26.000000,20.833333,0.000000,0.000000,0.153846,0.000000,10.500000,50.750000,-27.000000,23.853977,6.521739,25.000000,0.142857,0.000000,10.000000,48.700000,-32.000000,20.000000,11.111111,21.428571,0.142857,0.000000,6.000000,52.866667,-24.000000,0.000000
25%,41.801382,32.174571,65.633779,0.820099,2.333333,17.833333,67.169271,-12.950000,41.638742,32.047005,66.071278,0.835104,2.666667,17.888889,67.200568,-12.839912,41.880465,32.035841,65.279426,0.826212,2.352941,17.750000,67.208304,-13.070813,41.616138,32.080997,66.349778,0.831233,2.682416,17.896053,67.119444,-12.833333,0.000000
50%,43.916470,34.577782,68.871192,0.971042,3.200000,19.163333,69.818750,-11.346327,43.740437,34.361893,68.687503,0.968616,3.266667,19.214286,69.875893,-11.125000,44.099431,34.652809,68.730522,0.980659,3.181818,19.076923,69.937500,-11.428571,43.601487,34.214607,68.752964,0.970612,3.272727,19.181818,69.935000,-11.142857,0.000000
75%,46.024550,37.012899,71.999970,1.121413,4.125000,20.528638,72.757812,-9.728788,46.074320,36.868188,70.969299,1.117039,3.875000,20.454545,72.740469,-9.688702,46.120414,37.293508,71.885866,1.128894,4.125000,20.427198,72.769278,-9.692308,45.892020,36.760706,70.950944,1.113357,3.857143,20.528638,72.819231,-9.666667,1.000000
max,62.258755,56.250000,89.529915,2.079545,11.000000,32.000000,97.400000,10.000000,58.333333,58.823529,92.753623,2.000000,19.000000,28.000000,99.350000,3.500000,63.796296,59.343434,100.000000,2.111111,13.000000,30.500000,97.400000,7.000000,60.000000,63.636364,100.000000,2.267857,10.000000,32.000000,99.350000,12.000000,1.000000


In [19]:
final_data = get_data_reg()

4680it [00:10, 431.41it/s]00<?, ?it/s]
4635it [00:10, 434.14it/s]10<03:37, 10.87s/it]
4739it [00:10, 436.31it/s]21<03:24, 10.77s/it]
4821it [00:10, 438.51it/s]32<03:14, 10.82s/it]
5107it [00:11, 433.33it/s]43<03:05, 10.90s/it]
5227it [00:12, 432.22it/s]55<02:59, 11.22s/it]
5313it [00:12, 431.78it/s]07<02:52, 11.53s/it]
10427it [00:25, 412.55it/s]9<02:45, 11.79s/it]
10460it [00:25, 413.68it/s]5<03:29, 16.10s/it]
10433it [00:24, 428.74it/s]0<03:47, 18.99s/it]
10593it [00:24, 431.40it/s]34<03:47, 20.65s/it]
10718it [00:25, 428.30it/s]59<03:38, 21.86s/it]
10698it [00:26, 410.78it/s]24<03:25, 22.84s/it]
10708it [00:25, 424.54it/s]50<03:10, 23.82s/it]
10735it [00:25, 428.60it/s]15<02:49, 24.26s/it]
10744it [00:25, 415.45it/s]40<02:27, 24.52s/it]
10833it [00:29, 370.97it/s]06<02:04, 24.94s/it]
10499it [00:26, 400.74it/s]36<01:44, 26.23s/it]
7540it [00:17, 427.24it/s]:02<01:18, 26.24s/it]
10539it [00:25, 414.45it/s]19<00:47, 23.67s/it]
9400it [00:21, 428.16it/s]:45<00:24, 24.21s/it]
100%|█████

In [20]:
final_data.to_csv('computed_data/reg_avg_data.csv',index=False)

In [21]:
final_data = pd.read_csv('computed_data/reg_avg_data.csv')

In [22]:
final_data.shape

(161997, 37)